<a href="https://colab.research.google.com/github/amamra-abdenour/Deep_Learning_Python/blob/master/DeapLearning_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top= False,
                  input_shape=(150,150,3)
)
conv_base.summary()





58892288/58889256 [==============================] - 2s 0us/step


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
________________________________________________________

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2019-10-02 20:03:09--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   111MB/s    in 0.6s    

2019-10-02 20:03:10 (111 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
import zipfile
with zipfile.ZipFile('/tmp/cats_and_dogs_filtered.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [5]:
import os
import shutil

dataset_dir = '/content/cats_and_dogs_filtered/'
base_dir ='/content/cats_and_dogs_small/'
os.mkdir(base_dir)



FileExistsError: ignored

In [26]:
from os import listdir
print(len(listdir(dataset_dir+'train/cats/')))
listdir(dataset_dir+'train/cats/')

SyntaxError: ignored

In [0]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In [0]:
validation_dir_cats = os.path.join(validation_dir, 'cats')
os.mkdir(validation_dir_cats)

validation_dir_dogs = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dir_dogs)

In [0]:
test_dir_cats = os.path.join(test_dir, 'cats')
os.mkdir(test_dir_cats)

test_dir_dogs = os.path.join(test_dir, 'dogs')
os.mkdir(test_dir_dogs)


In [0]:
train_dir_cats = os.path.join(train_dir, 'cats')
os.mkdir(train_dir_cats)

train_dir_dogs = os.path.join(train_dir, 'dogs')
os.mkdir(train_dir_dogs)


In [0]:
frames = listdir(dataset_dir+'train/cats/')
for frame in frames:
  src = os.path.join(dataset_dir+'train/cats/', frame)
  dst = os.path.join(train_dir_cats,frame)
  shutil.copyfile(src,dst)

In [0]:
frames = listdir(dataset_dir+'train/dogs/')
for frame in frames:
  src = os.path.join(dataset_dir+'train/dogs/', frame)
  dst = os.path.join(train_dir_dogs,frame)
  shutil.copyfile(src,dst)

In [0]:
frames = listdir(dataset_dir+'validation/cats/')
for frame in frames:
  src = os.path.join(dataset_dir+'validation/cats/', frame)
  dst = os.path.join(validation_dir_cats,frame)
  shutil.copyfile(src,dst)

In [0]:
frames = listdir(dataset_dir+'validation/dogs/')
for frame in frames:
  src = os.path.join(dataset_dir+'validation/dogs/', frame)
  dst = os.path.join(validation_dir_dogs,frame)
  shutil.copyfile(src,dst)

In [25]:
#sanity check
print(len(listdir(train_dir_cats)))
print(len(listdir(train_dir_dogs)))
print(len(listdir(validation_dir_cats)))
print(len(listdir(validation_dir_dogs)))

1000
1000
500
500


In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size=20
def extract_features(directory, sample_count):
    features = np.zeros(shape = (sample_count,4, 4, 512))
    labels = np.zeros(shape =  (sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary'
    )
    i=0
    for input_batch, label_batch in generator:
      features_batch = conv_base.predict(input_batch)
      print(i * batch_size , (i+1) * batch_size )
      features[i * batch_size : (i + 1) * batch_size] = features_batch
      labels[i * batch_size : (i + 1) * batch_size] = label_batch
      i += 1
      if i * batch_size >= sample_count:
        break
    return features, labels


In [39]:
train_features, train_labels = extract_features(train_dir,20)
validation_features, validation_labels = extract_features(validation_dir,20)

Found 2000 images belonging to 2 classes.
0 20
Found 1000 images belonging to 2 classes.
0 20


In [42]:
print(train_features.shape)
print(train_labels.shape)
print(validation_features.shape)
print(validation_labels.shape)

(20, 4, 4, 512)
(20,)
(20, 4, 4, 512)
(20,)


In [0]:
train_features = np.reshape(train_features,(20, 4 * 4 * 512))
validation_features = np.reshape(validation_features,(20, 4 * 4 * 512))

In [55]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.losses import binary_crossentropy
from keras.optimizers import rmsprop
from keras.activations import relu, sigmoid

model = Sequential()
model.add(Dense(256, activation = relu, input_dim =(4*4*512)))
model.add(Dropout(0.5))
model.add(Dense(1,activation = sigmoid))

model.compile (optimizer = rmsprop(lr=1e-5),
               loss=binary_crossentropy,
               metrics=['acc'])

history = model.fit(train_features, train_labels, 
          epochs = 20, 
          batch_size = 20, 
          validation_data = (validation_features, validation_labels)
          )
model.save('VGG16_convbase_cats_dogs_1.h5')

Train on 20 samples, validate on 20 samples
Epoch 1/20
20/20 [==============================] - 1s 38ms/step - loss: 0.7670 - acc: 0.5500 - val_loss: 0.7684 - val_acc: 0.3000
Epoch 2/20
20/20 [==============================] - 0s 2ms/step - loss: 0.9059 - acc: 0.4000 - val_loss: 0.7497 - val_acc: 0.4000
Epoch 3/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5907 - acc: 0.6500 - val_loss: 0.7379 - val_acc: 0.4000
Epoch 4/20
20/20 [==============================] - 0s 2ms/step - loss: 0.7076 - acc: 0.6500 - val_loss: 0.7317 - val_acc: 0.5500
Epoch 5/20
20/20 [==============================] - 0s 2ms/step - loss: 0.6839 - acc: 0.6000 - val_loss: 0.7284 - val_acc: 0.5500
Epoch 6/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5126 - acc: 0.7500 - val_loss: 0.7262 - val_acc: 0.5500
Epoch 7/20
20/20 [==============================] - 0s 2ms/step - loss: 0.6607 - acc: 0.7000 - val_loss: 0.7243 - val_acc: 0.5500
Epoch 8/20
20/20 [===========================